In [1]:
%%spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1586491211808_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr
import pyspark.sql.functions as F
import pyspark.sql.types as T


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import GBTRegressor

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

#load from URL
#from pyspark import SparkFiles


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:

sc.install_pypi_package("pandas")
sc.install_pypi_package("matplotlib")
sc.install_pypi_package("seaborn")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


#from rdkit import Chem
#import deepchem as dc

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Bace analysis

In [6]:
url='https://github.com/jtwang1027/pyspark_aws/raw/master/bace.csv'
pd_df1=pd.read_csv(url)

#mol2vec data
url2='https://github.com/jtwang1027/pyspark_aws/raw/master/mol2vec.csv'
pd_df2=pd.read_csv(url2, header=None)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'pd' is not defined
Traceback (most recent call last):
NameError: name 'pd' is not defined



In [6]:
#concatenating word2vec matrix with original df
pd_df_comb=pd.concat([pd_df1,pd_df2],axis=1)

#rename columns to dodge issues with processing spark df using parentheses in column names


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'pd' is not defined
Traceback (most recent call last):
NameError: name 'pd' is not defined



In [7]:
#rename columns to dodge issues with processing spark df using parentheses in column names
pd_df1.columns=['mol', 'CID', 'Class', 'Model', 'pIC50'] + list(range(len(pd_df1.columns)-5))
#creating spark df with just original df
df=spark.createDataFrame(pd_df1)

#rename columns to dodge issues with processing spark df using parentheses in column names
pd_df_comb.columns=['mol', 'CID', 'Class', 'Model', 'pIC50'] + list(range(len(pd_df.columns)-5))
#creating spark df with both original df+ word2vec
df_w2v=spark.createDataFrame(pd_df_comb)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'pd_df1' is not defined
Traceback (most recent call last):
NameError: name 'pd_df1' is not defined



In [8]:
#feature columns, combines all input columns into 1 feature column with an array in each row
assembler= VectorAssembler( inputCols=df.columns[5:], outputCol="pre_scaled_features") 

#scale the data to have zero mean, unit sd
#outputCol:features, which makes it easier for  downstream modeling defaults
scaler= StandardScaler( inputCol="pre_scaled_features", outputCol="features", withStd= True, withMean=False)

#create pipeline composed of transform operations defined above
pipeline=Pipeline(stages=[assembler,scaler])

#fit and apply transformation to each spark df
scalerModel1 = pipeline.fit(df)
scaledData1 = scalerModel1.transform(df)

#scalerModel_w2v = pipeline.fit(df_w2v)
#scaledData_w2v = scalerModel1.transform(df_w2v)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'df' is not defined
Traceback (most recent call last):
NameError: name 'df' is not defined



In [86]:
#split into train /test based on column: Model
Xtrain= scaledData.filter((col('Model') == 'Train') | (col('Model') == 'Test'))
Xtest= scaledData.filter(col('Model') == 'Valid')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [87]:
print("Training rows: " + str( Xtrain.count()))
print("Testing rows: " + str( Xtest.count()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Training rows: 1468
Testing rows: 45

## Mol2vec

In [30]:
from rdkit import Chem
import deepchem as dc
#from mol2vec.features import mol2alt_sentence, mol2sentence, MolSentence, DfVec, sentences2vec
#from gensim.models import word2vec

feat= dc.feat.RDKitDescriptors()
arr= feat.featurize(mols)

mols=[Chem.MolFromSmiles(smile) for smile in smiles]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
import mol2vec

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

No module named 'mol2vec'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'mol2vec'



In [40]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/') 
sys.path.append('/home/hadoop/.local/lib/python3.7/site-packages') 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
Xtrain.select('MW').show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+
|               MW|
+-----------------+
|        431.56979|
|        657.81073|
|        591.74091|
|        591.67828|
|629.7128299999999|
+-----------------+
only showing top 5 rows

## Multiple linear regression

In [88]:

lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8,
                      featuresCol='features',labelCol='pIC50')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [89]:
#fit model
lrModel = lr.fit(Xtrain)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [90]:
#evaluate LR model
summary=lrModel.evaluate(Xtest)

print("RMSE from Linear Regression Model: {}".format(summary.rootMeanSquaredError))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

RMSE from Linear Regression Model: 1.5614018267777021

## Gradient-boosted tree regression

In [92]:
gbt_model = GBTRegressor(featuresCol="features", labelCol='pIC50',maxIter=10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [93]:
gbt=gbt_model.fit(Xtrain)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [94]:
gbt_pred= gbt.transform(Xtest) #returns all prediction and all features (from Xtrain)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [95]:
gbt_pred.select(['pIC50','prediction']).show(45)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+
|             pIC50|        prediction|
+------------------+------------------+
|         6.7212462| 6.571227146147915|
|         10.522879| 8.544735011305649|
| 9.522878599999999| 8.544735011305649|
| 9.522878599999999| 8.544735011305649|
|         9.1549015|  8.51161253202241|
|               9.0|  8.51161253202241|
|               9.0| 8.490736475282192|
|               9.0| 8.544735011305649|
|               9.0| 8.207344765893756|
| 8.958607699999998| 8.490736475282192|
|         8.7695513| 8.141723693627734|
| 8.744727099999999| 8.467202095896193|
| 8.522878599999999|  8.21659828639141|
| 8.522878599999999| 8.207344765893756|
|         8.5086384|  8.51161253202241|
|         8.2676058| 7.919328049266717|
|         8.2218485| 8.182586551827047|
|         7.9586072| 7.964073064792522|
|         7.8860564| 8.490736475282192|
|         7.6989698| 8.162599750367951|
|         7.6020598| 8.490736475282192|
|         7.6003261| 8.510723276741286|


In [96]:
#Defining evaluator (gbt doesn't have "evaluate" method)
evaluator=RegressionEvaluator(labelCol="pIC50", predictionCol="prediction",metricName="rmse")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [97]:
#evaluating GBT model

rmse_gbt=evaluator.evaluate(gbt_pred)

print("RMSE from GBT Model: {}".format(rmse_gbt))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

RMSE from GBT Model: 1.2923119006069714

## logP analysis

In [8]:
df = (
    spark.read.
    format('csv').
    option('header', 'true').
    option('inferSchema', 'true').
    load('s3://jw-spark/logP_dataset.csv')
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- smiles: string (nullable = true)
 |-- logp: double (nullable = true)

In [9]:
df.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+
|              smiles|logp|
+--------------------+----+
|C[C@H]([C@@H](C)C...| 2.3|
|           C(C=CBr)N| 0.3|
|           CCC(CO)Br| 1.3|
|[13CH3][13CH2][13...| 2.0|
|             CCCOCCP| 0.6|
|        C(C(F)(F)F)F| 1.7|
|[2H]C([2H])C(C)(C)Cl| 1.8|
|           CCCC(CI)O| 2.0|
|        CCCCCC[CH+]C| 3.9|
|           C(CO)NCCO|-1.4|
+--------------------+----+
only showing top 10 rows

In [11]:
from rdkit import Chem

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

No module named 'rdkit'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'rdkit'



In [33]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import PCA
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
import pandas as pd

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
df = (
    spark.read.
    format('csv').
    option('header', 'false').
    option('inferSchema', 'true').
    load('s3://jw-spark/sonar.txt')
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _c0: double (nullable = true)
 |-- _c1: double (nullable = true)
 |-- _c2: double (nullable = true)
 |-- _c3: double (nullable = true)
 |-- _c4: double (nullable = true)
 |-- _c5: double (nullable = true)
 |-- _c6: double (nullable = true)
 |-- _c7: double (nullable = true)
 |-- _c8: double (nullable = true)
 |-- _c9: double (nullable = true)
 |-- _c10: double (nullable = true)
 |-- _c11: double (nullable = true)
 |-- _c12: double (nullable = true)
 |-- _c13: double (nullable = true)
 |-- _c14: double (nullable = true)
 |-- _c15: double (nullable = true)
 |-- _c16: double (nullable = true)
 |-- _c17: double (nullable = true)
 |-- _c18: double (nullable = true)
 |-- _c19: double (nullable = true)
 |-- _c20: double (nullable = true)
 |-- _c21: double (nullable = true)
 |-- _c22: double (nullable = true)
 |-- _c23: double (nullable = true)
 |-- _c24: double (nullable = true)
 |-- _c25: double (nullable = true)
 |-- _c26: double (nullable = true)
 |-- _c27: double (nullable = tru

In [51]:
df=df.withColumnRenamed('_c60', 'raw_label') #rename last column logP

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['_c0', '_c1', '_c2', '_c3', '_c4', '_c5', '_c6', '_c7', '_c8', '_c9', '_c10', '_c11', '_c12', '_c13', '_c14', '_c15', '_c16', '_c17', '_c18', '_c19', '_c20', '_c21', '_c22', '_c23', '_c24', '_c25', '_c26', '_c27', '_c28', '_c29', '_c30', '_c31', '_c32', '_c33', '_c34', '_c35', '_c36', '_c37', '_c38', '_c39', '_c40', '_c41', '_c42', '_c43', '_c44', '_c45', '_c46', '_c47', '_c48', '_c49', '_c50', '_c51', '_c52', '_c53', '_c54', '_c55', '_c56', '_c57', '_c58', '_c59', 'raw_label']

In [53]:

train, test = df.randomSplit([0.7, 0.3])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:

transformer = VectorAssembler(
    inputCols=['_c%01d' % i for i in range(60)],
    outputCol="raw_features"
)
standardizer = StandardScaler(
    withMean=True, 
    withStd=True, 
    inputCol='raw_features', 
    outputCol='features'
)
indexer = StringIndexer(
    inputCol="raw_label", 
    outputCol="label"
)
pca = PCA(
    k=5, 
    inputCol="features", 
    outputCol="pca"
)
lr = LogisticRegression(
    featuresCol='features', 
    labelCol='label'
)

pipeline = Pipeline(stages=[transformer, standardizer, indexer, pca, lr])
model = pipeline.fit(train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
 prediction = model.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
score = prediction.select(['label', 'prediction'])
score.show(n=score.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       1.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  0.0|       1.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
+-----+----------+

# 